# STAT 207 Homework 11 [25 points]

## Regularization Models for Linear Relationships

Due: Thursday, May 1, end of day (11:59 pm CT)

Late submissions accepted until Friday, May 2 at noon

<hr>

## Imports 

Run the following code cell to import the necessary packages into the file.  You may import additional packages, as needed for this assignment.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

## The Data

With available climate data dating back many decades and the prevalence of climate change, humans are looking to understand exactly how different features of the climate affect the temperatures globally.  For this assignment, we will look to understand how the **global temperature** fluctuates based on other environmental features.

We will use various atmospheric and temperature measures over 309 months from 1983 to 2008, with the following variables:

- **Year**: the observation year
- **Month**: the observation month, recorded with numbers 1 to 12
- **MEI**: Multivariate El Nino Southern Oscillation Index (MEI), measuring the affects of the El Nino weather pattern
- **CO2**: atmospheric concentration of carbon dioxide (in ppmv, parts per million by volume)
- **CH4**: atmospheric concentration of methane (in ppmv)
- **N2O**: atmospheric concentration of nitrous oxide (in ppmv)
- **CFC-11**: atmospheric concentration of CCl3F or trichlorofluoromethane (in ppbv, parts per billion by volume)
- **CFC-12**: atmospheric concentration of CCl2F2 or dichlorodifluoromethane (in ppbv)
- **TSI**: the total solar irradiance (TSI) (in W/m2), measuring the rate at which the sun's energy is deposited per unit area.
- **Aerosols**: the mean stratospheric aerosol optical depth at 500 nm, a measure associated with volcanic activity
- **Temp**: the difference in the average global temperature for that month (in Celsius) and a reference value

The ESRL/NOAA Physical Sciences Division reports the MEI; atmospheric concentrations are measured by the ESRL/NOAA Global Monitoring Division; the SOLARIS-HEPPA project website provides the TSI; the Godard Institute for Space Studies at NASA reports the Aerosols; and the Climatic Research Unit at the University of East Anglia reports the Temp.

Run the code in the cell below to read in the cleaned data for this document.  The data is saved as `df` with this code.  

In [3]:
df = pd.read_csv('climate_change.csv')
df_train = df[df['Year'] <= 2006]
df_test = df[df['Year'] >= 2007]
X_train = df_train.drop(['Year', 'Month', 'Temp'], axis = 1)
X_test = df_test.drop(['Year', 'Month', 'Temp'], axis = 1)
y_train = df_train['Temp']
y_test = df_test['Temp']

df

,Year,Month,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols,Temp
0,1983,5,2.556,345.96,1638.59,303.677,191.324,350.113,1366.1024,0.0863,0.109
1,1983,6,2.167,345.52,1633.71,303.746,192.057,351.848,1366.1208,0.0794,0.118
2,1983,7,1.741,344.15,1633.22,303.795,192.818,353.725,1366.2850,0.0731,0.137
3,1983,8,1.130,342.25,1631.35,303.839,193.602,355.633,1366.4202,0.0673,0.176
4,1983,9,0.428,340.17,1648.40,303.901,194.392,357.465,1366.2335,0.0619,0.149
...,...,...,...,...,...,...,...,...,...,...,...
303,2008,8,-0.266,384.15,1779.88,321.405,244.200,535.072,1365.6570,0.0036,0.407
304,2008,9,-0.643,383.09,1795.08,321.529,244.083,535.048,1365.6647,0.0043,0.378
305,2008,10,-0.780,382.99,1814.18,321.796,244.080,534.927,1365.6759,0.0046,0.440
306,2008,11,-0.621,384.13,1812.37,322.013,244.225,534.906,1365.7065,0.0048,0.394


## 1. Summarize Data [1.5 points]

Above, we set aside a training data.  We didn't randomly select our training and test set; instead, imagine that we fit a model using the available data in 2006 in our training data.  We'll then use the data that we collect in the following two years as the test set to evaluate this model.

As defined in our X_train and X_test above, our response variable for this assignment with be **Temp**.  We'll use all variables except the **Year** and **Month** as our predictor variables.

**a)** Scale our predictor variables in the training data.

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns, index= X_train.index)

In [5]:
mean = X_train.mean()
largest_mean = mean.max()
smallest_mean = mean.min()

largest_mean, smallest_mean

(1745.8414788732393, 0.017720774647887325)

In [6]:
mean = X_train_scaled.mean()
largest_mean = mean.max()
smallest_mean = mean.min()

largest_mean, smallest_mean

(2.5019110414088033e-15, -4.33681259917802e-13)

**b)** Now, we want to be sure that we also scale our test data, using the same scaling as applied to our training data.  Apply your scaling algorithm from **part a** to the test data.  

*Note*: This does not include re-fitting your scaling process.  You will re-use your scaling process from **part a**, simply transforming your test data with the same scaling process.

In [7]:
X_test_scaled = scaler.transform(X_test)

X_test_scaled = pd.DataFrame(X_test_scaled, 
                             columns=X_test.columns)

In [8]:
X_test_scaled.describe()

,MEI,CO2,CH4,N2O,CFC-11,CFC-12,TSI,Aerosols
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,-0.917795,2.036889,1.121218,1.984680,-0.314865,0.720071,-0.982834,-0.455588
std,0.690182,0.179041,0.250031,0.114226,0.065486,0.024218,0.079206,0.020206
min,-2.130303,1.706352,0.571373,1.826219,-0.401280,0.690306,-1.109496,-0.487991
25%,-1.493181,1.895939,1.020765,1.873744,-0.375720,0.693835,-1.064123,-0.471303
50%,-0.940110,2.023790,1.145081,2.020740,-0.301021,0.725222,-0.965827,-0.454614
75%,-0.407244,2.164996,1.265504,2.052897,-0.281332,0.732229,-0.936619,-0.441264
max,0.681118,2.371878,1.499001,2.215683,-0.196418,0.763259,-0.848371,-0.411225


## 2. Fitting A Model [1 point]

Fit a LASSO model with $\lambda = 0.06$ to the training data, including all variables except the year and month variables.  Print the coefficients for this model.

In [9]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=0.06)

lasso_model.fit(X_train_scaled, y_train)

coeff_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Lasso Coefficient': lasso_model.coef_})

print(coeff_df)


   Variable  Lasso Coefficient
0       MEI           0.000000
1       CO2           0.079893
2       CH4           0.000000
3       N2O           0.002758
4    CFC-11           0.000000
5    CFC-12           0.000000
6       TSI           0.000000
7  Aerosols          -0.000000


## 3. Picking a Best Model [2.5 points]

Instead of using a LASSO model, we decide that we'd rather move forward with a **ridge regression** model.  We don't know which $\lambda$ to use for this model, so let's explore which value of $\lambda$ might be most appropriate for a ridge regression model.

To do this, we'll explore $\lambda$ values between 0.05 and 1 exploring by every 0.05.  We can do this with code using:

`for m in range(1, 21):`

`alph = m / 20`

The following code sets up the folds for cross-validation.

In [10]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cross_val = KFold(n_splits = 10, shuffle = True, random_state = 202505)

**a)** Use 10-fold cross-validation to explore the $R^2$ values for each of the $\lambda$s as defined above.

In [11]:
best_r2 = -float('inf')
best_lambda = None

for m in range(1, 21):
    alph = m / 20
    ridge_model = Ridge(alpha=alph)
    r2_scores = cross_val_score(ridge_model, X_train_scaled, y_train, cv=cross_val, scoring='r2')
    mean_r2 = r2_scores.mean()
    
    print(f"lambda = {alph}, mean R² = {mean_r2}")

    if mean_r2 > best_r2:
        best_r2 = mean_r2
        best_lambda = alph

print('----')
print(f"Best lambda: {best_lambda}, with mean R²: {best_r2}")


lambda = 0.05, mean R² = 0.723573522724511
lambda = 0.1, mean R² = 0.7236379854142208
lambda = 0.15, mean R² = 0.723675636421907
lambda = 0.2, mean R² = 0.7236913449221215
lambda = 0.25, mean R² = 0.7236890920412968
lambda = 0.3, mean R² = 0.7236721465994678
lambda = 0.35, mean R² = 0.7236432027687879
lambda = 0.4, mean R² = 0.7236044886428361
lambda = 0.45, mean R² = 0.7235578524133405
lambda = 0.5, mean R² = 0.7235048311834638
lambda = 0.55, mean R² = 0.7234467062250581
lambda = 0.6, mean R² = 0.7233845475843014
lambda = 0.65, mean R² = 0.7233192502671459
lambda = 0.7, mean R² = 0.72325156373054
lambda = 0.75, mean R² = 0.7231821160229291
lambda = 0.8, mean R² = 0.7231114336261297
lambda = 0.85, mean R² = 0.7230399578271806
lambda = 0.9, mean R² = 0.7229680582762829
lambda = 0.95, mean R² = 0.7228960442530267
lambda = 1.0, mean R² = 0.7228241740585505
----
Best lambda: 0.2, with mean R²: 0.7236913449221215


**b)** Print the $R^2$ values for each of the individual folds of your optimal $\lambda$.

In [12]:
best_ridge = Ridge(alpha=0.2)

individual_r2_scores = cross_val_score(best_ridge, X_train_scaled, y_train, cv=cross_val, scoring='r2')
print(individual_r2_scores)

[0.6847359  0.58854956 0.77279909 0.73198128 0.70566594 0.84894601
 0.67059818 0.68169537 0.84897019 0.70297192]


**c)** Repeat this process for a different set of 10-folds, using a random state of your choosing.  Determine which $\lambda$ results in the optimal mean $R^2$, similar to what you did above.

In [13]:
new_cross_val = KFold(n_splits=10, shuffle=True, random_state=42)

best_r2_new = -float('inf')
best_lambda_new = None

for m in range(1, 21):
    alph = m / 20
    ridge_model = Ridge(alpha=alph)
    r2_scores = cross_val_score(ridge_model, X_train_scaled, y_train, cv=new_cross_val, scoring='r2')
    mean_r2 = r2_scores.mean()
    
    print(f"lambda = {alph}, mean R² = {mean_r2}")
    
    if mean_r2 > best_r2_new:
        best_r2_new = mean_r2
        best_lambda_new = alph

print("----")
print(f"Best lambda from new folds: {best_lambda_new}, with mean R²: {best_r2_new}")

lambda = 0.05, mean R² = 0.7232656383174255
lambda = 0.1, mean R² = 0.7233045745085349
lambda = 0.15, mean R² = 0.7233151249321594
lambda = 0.2, mean R² = 0.7233026679712625
lambda = 0.25, mean R² = 0.7232715858015711
lambda = 0.3, mean R² = 0.7232254632258333
lambda = 0.35, mean R² = 0.7231672431748974
lambda = 0.4, mean R² = 0.7230993491528739
lambda = 0.45, mean R² = 0.7230237822696282
lambda = 0.5, mean R² = 0.722942198594025
lambda = 0.55, mean R² = 0.7228559711635536
lambda = 0.6, mean R² = 0.7227662399539017
lambda = 0.65, mean R² = 0.7226739523436766
lambda = 0.7, mean R² = 0.7225798960329421
lambda = 0.75, mean R² = 0.7224847259384698
lambda = 0.8, mean R² = 0.7223889862568987
lambda = 0.85, mean R² = 0.7222931286328647
lambda = 0.9, mean R² = 0.7221975271732184
lambda = 0.95, mean R² = 0.7221024908964888
lambda = 1.0, mean R² = 0.7220082740882204
----
Best lambda from new folds: 0.15, with mean R²: 0.7233151249321594


## 4. Evaluating Our Best Model [1 point]

Refit the model with the optimal $\lambda$ found in Question **3b** to the full training data.  Print the resulting coefficients.

In [14]:
ridge_best_model = Ridge(alpha=0.2)
ridge_best_model.fit(X_train_scaled, y_train)

coef_df = pd.DataFrame({
    'Variable': X_train.columns,
    'Coefficient': ridge_best_model.coef_
})

print(coef_df)


   Variable  Coefficient
0       MEI     0.059469
1       CO2     0.073007
2       CH4     0.006589
3       N2O    -0.064418
4    CFC-11    -0.124992
5    CFC-12     0.199807
6       TSI     0.036865
7  Aerosols    -0.046174


In [16]:
from sklearn.metrics import r2_score

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

ridge = Ridge(alpha=0.2)
ridge.fit(X_train_scaled, y_train)

y_test_pred = ridge.predict(X_test_scaled)
test_r2 = r2_score(y_test, y_test_pred)

print("R² score on test set:", test_r2)


R² score on test set: 0.15458483572756732


Remember to keep all your cells and hit the save icon above periodically to checkpoint (save) your results on your local computer. Once you are satisified with your results restart the kernel and run all (Kernel -> Restart & Run All). **Make sure nothing has changed**. Checkpoint and exit (File -> Save and Checkpoint + File -> Close and Halt). Follow the instructions on the Homework 11 Canvas Assignment to submit your notebook to GitHub.